# Vibe Coding: Real-World Data Cleaning Challenge

## The Mission

You're a Data Analyst at **TechSalary Insights**. Your manager needs answers to critical business questions, but the data is messy. Your job is to clean it and provide accurate insights.

**The catch:** You must figure out how to clean the data yourself. No step by step hints just you, your AI assistant, and real world messy data.

---

## The Dataset: Ask A Manager Salary Survey 2021

**Location:** `../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`

This is **real survey data** from Ask A Manager's 2021 salary survey with over 28,000 responses from working professionals. The data comes from this survey: https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html

**Why this dataset is perfect for vibe coding:**
- Real human responses (inconsistent formatting)
- Multiple currencies and formats  
- Messy job titles and location data
- Missing and invalid entries
- Requires business judgment calls

---

## Your Business Questions

Answer these **exact questions** with clean data. There's only one correct answer for each:

### Core Questions (Required):
1. **What is the median salary for Software Engineers in the United States?** 
2. **Which US state has the highest average salary for tech workers?**
3. **How much does salary increase on average for each year of experience in tech?**
4. **Which industry (besides tech) has the highest median salary?**

### Bonus Questions (If time permits):
5. **What's the salary gap between men and women in tech roles?**
6. **Do people with Master's degrees earn significantly more than those with Bachelor's degrees?**

**Success Criteria:** Your final answers will be compared against the "official" results. Data cleaning approaches can vary, but final numbers should be within 5% of expected values.


---
# Your Work Starts Here

## Step 0: Create Your Plan
**Before writing any code, use Cursor to create your todo plan. Then paste it here:**

```
## My Data Cleaning Plan

### Phase 1: Data Understanding & Preparation
- [x] Data Loading and Initial Exploration - Load TSV file and understand structure
- [ ] Data Quality Assessment - Identify missing values, inconsistencies, outliers  
- [ ] Salary Data Cleaning - Handle currencies, ranges, invalid entries
- [ ] Job Title and Industry Standardization - Define tech roles and categories
- [ ] Geographic Data Cleaning - Standardize US states and locations
- [ ] Experience and Education Data Processing - Convert to analyzable formats

### Phase 2: Core Business Questions
- [ ] Answer Core Question 1 - Median salary for Software Engineers in US
- [ ] Answer Core Question 2 - US state with highest average tech salary
- [ ] Answer Core Question 3 - Salary increase per year of experience in tech
- [ ] Answer Core Question 4 - Highest paying non-tech industry

### Phase 3: Advanced Analysis & Validation
- [ ] Answer Bonus Questions - Gender pay gaps, education impact
- [ ] Results Validation and Summary - Verify findings and document insights

**Key Challenges Expected:**
- Real-world messiness: Human survey responses with inconsistent formatting
- Currency variations: Multiple currencies need USD conversion
- Job title ambiguity: Various ways to describe similar roles
- Geographic inconsistencies: State names, abbreviations, remote work
- Salary outliers: Unrealistic values requiring judgment calls
```

## Step 1: Data Loading and Exploration

Start here! Load the dataset and get familiar with what you're working with.


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Step 2: Data Cleaning

Now let's tackle the messy data! Starting with salary standardization and currency conversion.

In [2]:
# SALARY DATA CLEANING
print("=" * 60)
print("SALARY DATA CLEANING")
print("=" * 60)

# Step 1: Clean salary strings and convert to numeric
print("Step 1: Converting salary strings to numeric...")

# Create a copy to work with
df_clean = df.copy()

# Function to clean salary strings
def clean_salary_string(salary_str):
    """Clean salary string and convert to numeric"""
    if pd.isna(salary_str):
        return np.nan
    
    # Convert to string and handle various formats
    salary_str = str(salary_str).strip()
    
    # Remove common non-numeric characters
    salary_clean = salary_str.replace(',', '').replace('$', '').replace(' ', '')
    
    # Handle ranges (take the midpoint)
    if '-' in salary_clean:
        parts = salary_clean.split('-')
        if len(parts) == 2:
            try:
                low = float(parts[0])
                high = float(parts[1])
                return (low + high) / 2
            except:
                pass
    
    # Try to convert to float
    try:
        return float(salary_clean)
    except:
        return np.nan

# Clean salary column
df_clean['salary_numeric'] = df_clean['salary'].apply(clean_salary_string)

# Check results
print(f"Original salary samples: {df['salary'].head().tolist()}")
print(f"Cleaned salary samples: {df_clean['salary_numeric'].head().tolist()}")
print(f"Successfully converted: {df_clean['salary_numeric'].notna().sum():,} out of {len(df_clean):,} records")
print(f"Failed conversions: {df_clean['salary_numeric'].isna().sum():,}")

# Check for extreme outliers
print(f"\nSalary statistics:")
print(df_clean['salary_numeric'].describe())

SALARY DATA CLEANING
Step 1: Converting salary strings to numeric...


NameError: name 'df' is not defined

In [ ]:
# Step 2: Currency Conversion to USD
print("\nStep 2: Converting currencies to USD...")

# Approximate exchange rates (using 2021 averages)
exchange_rates = {
    'USD': 1.0,
    'CAD': 0.79,    # 1 CAD = 0.79 USD
    'GBP': 1.38,    # 1 GBP = 1.38 USD  
    'EUR': 1.18,    # 1 EUR = 1.18 USD
    'AUD/NZD': 0.75, # Average for AUD/NZD
    'CHF': 1.10,    # 1 CHF = 1.10 USD
    'SEK': 0.12,    # 1 SEK = 0.12 USD
    'JPY': 0.009,   # 1 JPY = 0.009 USD
    'ZAR': 0.067,   # 1 ZAR = 0.067 USD
    'HKD': 0.13,    # 1 HKD = 0.13 USD
    'Other': 1.0    # Assume USD for "Other" as most are likely USD
}

# Convert salaries to USD
def convert_to_usd(row):
    """Convert salary to USD based on currency"""
    salary = row['salary_numeric']
    currency = row['currency']
    
    if pd.isna(salary):
        return np.nan
        
    if currency in exchange_rates:
        return salary * exchange_rates[currency]
    else:
        # For currencies not in our list, assume USD
        return salary

df_clean['salary_usd'] = df_clean.apply(convert_to_usd, axis=1)

print("Currency conversion summary:")
print(df_clean.groupby('currency')['salary_usd'].agg(['count', 'mean', 'median']).round(0))

print(f"\nUSD salary statistics after conversion:")
print(df_clean['salary_usd'].describe())


Step 2: Converting currencies to USD...
Currency conversion summary:
          count       mean    median
currency                            
AUD/NZD     504    79322.0   71250.0
CAD        1673  2898507.0   59566.0
CHF          37   145774.0  114400.0
EUR         643    72485.0   63720.0
GBP        1591    68865.0   55200.0
HKD           4    61978.0   45630.0
JPY          23    54447.0   43200.0
Other       160  8675873.0  251500.0
SEK          37    67344.0   58032.0
USD       23374    97041.0   78000.0
ZAR          16    36760.0   31590.0

USD salary statistics after conversion:
count    2.806200e+04
mean     3.104445e+05
std      2.880654e+07
min      0.000000e+00
25%      5.400000e+04
50%      7.500000e+04
75%      1.071460e+05
max      4.740055e+09
Name: salary_usd, dtype: float64


In [ ]:
# Step 3: Handle Outliers and Problematic Salaries
print("\nStep 3: Handling outliers and unrealistic salaries...")

# Identify extreme outliers
print("Extreme salary outliers:")
extreme_high = df_clean[df_clean['salary_usd'] > 1000000]  # Over $1M
extreme_low = df_clean[df_clean['salary_usd'] <= 1000]     # Under $1K

print(f"Salaries over $1M: {len(extreme_high)}")
if len(extreme_high) > 0:
    print("Sample high outliers:")
    print(extreme_high[['salary', 'salary_usd', 'currency', 'job_title', 'country']].head())

print(f"\nSalaries under $1K: {len(extreme_low)}")
if len(extreme_low) > 0:
    print("Sample low outliers:")
    print(extreme_low[['salary', 'salary_usd', 'currency', 'job_title', 'country']].head())

# Define reasonable salary bounds for 2021
# Based on typical salary ranges: $10K minimum (part-time), $500K maximum (executive level)
min_reasonable_salary = 10000
max_reasonable_salary = 500000

# Create cleaned salary column
df_clean['salary_usd_clean'] = df_clean['salary_usd'].copy()

# Set unrealistic salaries to NaN
unrealistic_mask = (df_clean['salary_usd_clean'] < min_reasonable_salary) | (df_clean['salary_usd_clean'] > max_reasonable_salary)
df_clean.loc[unrealistic_mask, 'salary_usd_clean'] = np.nan

print(f"\nSalary cleaning results:")
print(f"Original records: {len(df_clean):,}")
print(f"Records with realistic salaries: {df_clean['salary_usd_clean'].notna().sum():,}")
print(f"Removed as unrealistic: {unrealistic_mask.sum():,}")
print(f"Percentage kept: {(df_clean['salary_usd_clean'].notna().sum() / len(df_clean) * 100):.1f}%")

print(f"\nCleaned salary statistics:")
print(df_clean['salary_usd_clean'].describe())


Step 3: Handling outliers and unrealistic salaries...
Extreme salary outliers:
Salaries over $1M: 48
Sample high outliers:
           salary   salary_usd currency           job_title    country
603     1,080,000    1080000.0    Other          Exec admin  Argentina
2124    3,000,000    3000000.0      USD      Owner and CEO         USA
3162    1,400,000    1400000.0    Other         IT Director    Denmark
3605  102,000,000  102000000.0      USD  Operations Manager   Colombia
3937    1,150,000    1150000.0    Other      Senior Manager     Norway

Salaries under $1K: 114
Sample low outliers:
     salary  salary_usd currency                  job_title        country
97       58        58.0      USD     Quality Assurance Lead            USA
166      35        41.3      EUR     occupational therapist        finland
895      38        38.0      USD  Special Education Teacher            USA
968      61        61.0      USD            Managing Editor  United States
1607    130       130.0      

In [ ]:
# GEOGRAPHIC DATA CLEANING
print("\n" + "=" * 60)
print("GEOGRAPHIC DATA CLEANING")
print("=" * 60)

# Step 4: Standardize Country Names (Focus on US)
print("Step 4: Standardizing country names...")

def standardize_country(country_str):
    """Standardize country names, especially US variations"""
    if pd.isna(country_str):
        return np.nan
    
    country_clean = str(country_str).strip().lower()
    
    # US variations
    us_variations = [
        'us', 'usa', 'united states', 'united states of america', 
        'u.s.', 'u.s.a', 'america', 'united states america',
        'united states of american', 'usa ', 'unites states'
    ]
    
    if any(us_var in country_clean for us_var in us_variations):
        return 'United States'
    
    # UK variations
    uk_variations = ['uk', 'united kingdom', 'great britain', 'britain', 'england', 'scotland', 'wales']
    if any(uk_var in country_clean for uk_var in uk_variations):
        return 'United Kingdom'
        
    # Canada
    if 'canada' in country_clean:
        return 'Canada'
        
    # Australia
    if 'australia' in country_clean:
        return 'Australia'
        
    # For others, title case the original
    return str(country_str).strip().title()

df_clean['country_clean'] = df_clean['country'].apply(standardize_country)

print("Country standardization results:")
top_countries = df_clean['country_clean'].value_counts().head(10)
print(top_countries)

# Focus on US data for state cleaning
us_data = df_clean[df_clean['country_clean'] == 'United States'].copy()
print(f"\nUS records: {len(us_data):,} out of {len(df_clean):,} total ({len(us_data)/len(df_clean)*100:.1f}%)")


GEOGRAPHIC DATA CLEANING
Step 4: Standardizing country names...
Country standardization results:
country_clean
United States     23493
Canada             1679
United Kingdom     1578
Germany             195
Ireland             124
New Zealand         123
France               68
Netherlands          57
Spain                49
Sweden               41
Name: count, dtype: int64

US records: 23,493 out of 28,062 total (83.7%)


In [ ]:
# Step 5: Clean US State Data
print("\nStep 5: Cleaning US state data...")

# State abbreviation mapping
state_mapping = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming',
    'DC': 'District of Columbia'
}

def clean_state(state_str):
    """Clean and standardize US state names"""
    if pd.isna(state_str):
        return np.nan
    
    state_clean = str(state_str).strip()
    
    # Check if it's an abbreviation
    if state_clean.upper() in state_mapping:
        return state_mapping[state_clean.upper()]
    
    # Check for full state names (handle case variations)
    state_title = state_clean.title()
    if state_title in state_mapping.values():
        return state_title
        
    # Handle some common variations
    state_lower = state_clean.lower()
    if 'washington' in state_lower and ('dc' in state_lower or 'district' in state_lower):
        return 'District of Columbia'
    
    # Remote work indicators
    remote_indicators = ['remote', 'work from home', 'wfh', 'telecommute', 'virtual']
    if any(indicator in state_lower for indicator in remote_indicators):
        return 'Remote'
        
    # Return cleaned version if not found
    return state_clean

# Apply state cleaning to US data
us_data['state_clean'] = us_data['state'].apply(clean_state)

print("US State cleaning results:")
print(f"US records with state data: {us_data['state_clean'].notna().sum():,}")
print(f"Missing state data: {us_data['state_clean'].isna().sum():,}")

print("\nTop 15 US states by record count:")
state_counts = us_data['state_clean'].value_counts().head(15)
print(state_counts)

# Add state data back to main dataframe
df_clean['state_clean'] = df_clean['state'].apply(clean_state)


Step 5: Cleaning US state data...
US State cleaning results:
US records with state data: 22,905
Missing state data: 588

Top 15 US states by record count:
state_clean
California              2592
New York                2166
Massachusetts           1514
Texas                   1260
Illinois                1207
Washington              1177
District of Columbia     979
Pennsylvania             939
Virginia                 781
Minnesota                715
Ohio                     653
Colorado                 630
Oregon                   621
North Carolina           599
Maryland                 559
Name: count, dtype: int64


In [ ]:
# EXPERIENCE AND EDUCATION CLEANING
print("\n" + "=" * 60)
print("EXPERIENCE AND EDUCATION CLEANING")
print("=" * 60)

# Step 6: Convert Experience Ranges to Numeric
print("Step 6: Converting experience ranges to numeric...")

def experience_to_numeric(exp_str):
    """Convert experience range to numeric midpoint"""
    if pd.isna(exp_str):
        return np.nan
    
    exp_clean = str(exp_str).strip().lower()
    
    # Handle specific ranges
    if '1 year or less' in exp_clean:
        return 0.5
    elif '2 - 4' in exp_clean:
        return 3
    elif '5-7' in exp_clean:
        return 6
    elif '8 - 10' in exp_clean:
        return 9
    elif '11 - 20' in exp_clean:
        return 15
    elif '21 - 30' in exp_clean:
        return 25
    elif '31 - 40' in exp_clean:
        return 35
    elif '41 years or more' in exp_clean:
        return 45
    else:
        return np.nan

df_clean['total_experience_numeric'] = df_clean['total_experience'].apply(experience_to_numeric)
df_clean['field_experience_numeric'] = df_clean['field_experience'].apply(experience_to_numeric)

print("Experience conversion results:")
print(f"Total experience - converted: {df_clean['total_experience_numeric'].notna().sum():,}")
print(f"Field experience - converted: {df_clean['field_experience_numeric'].notna().sum():,}")

print("\nExperience statistics:")
print("Total experience:", df_clean['total_experience_numeric'].describe())
print("Field experience:", df_clean['field_experience_numeric'].describe())

# Step 7: Standardize Education Levels
print("\nStep 7: Standardizing education levels...")

def standardize_education(edu_str):
    """Standardize education levels"""
    if pd.isna(edu_str):
        return np.nan
    
    edu_clean = str(edu_str).strip().lower()
    
    if 'high school' in edu_clean:
        return 'High School'
    elif 'some college' in edu_clean:
        return 'Some College'
    elif 'college degree' in edu_clean or 'bachelor' in edu_clean:
        return "Bachelor's Degree"
    elif 'master' in edu_clean:
        return "Master's Degree"
    elif 'phd' in edu_clean or 'doctoral' in edu_clean:
        return 'PhD/Doctoral'
    elif 'professional' in edu_clean:
        return 'Professional Degree'
    else:
        return 'Other'

df_clean['education_clean'] = df_clean['education'].apply(standardize_education)

print("Education standardization results:")
print(df_clean['education_clean'].value_counts())


EXPERIENCE AND EDUCATION CLEANING
Step 6: Converting experience ranges to numeric...
Experience conversion results:
Total experience - converted: 28,062
Field experience - converted: 28,062

Experience statistics:
Total experience: count    28062.000000
mean        12.768317
std          8.048750
min          0.500000
25%          6.000000
50%         15.000000
75%         15.000000
max         45.000000
Name: total_experience_numeric, dtype: float64
Field experience: count    28062.000000
mean         9.386234
std          7.092610
min          0.500000
25%          3.000000
50%          6.000000
75%         15.000000
max         45.000000
Name: field_experience_numeric, dtype: float64

Step 7: Standardizing education levels...
Education standardization results:
education_clean
Bachelor's Degree      13519
Master's Degree         8865
Some College            2067
PhD/Doctoral            1426
Professional Degree     1324
High School              639
Name: count, dtype: int64


In [ ]:
# JOB TITLE AND TECH WORKER IDENTIFICATION
print("\n" + "=" * 60)
print("JOB TITLE AND TECH WORKER IDENTIFICATION")
print("=" * 60)

# Step 8: Identify Tech Workers and Software Engineers
print("Step 8: Identifying tech workers and software engineers...")

def identify_software_engineer(job_title):
    """Identify if job title is Software Engineer"""
    if pd.isna(job_title):
        return False
    
    title_lower = str(job_title).lower()
    
    # Software Engineer variations
    se_keywords = [
        'software engineer', 'software developer', 'software dev',
        'swe', 'sr. software engineer', 'senior software engineer',
        'junior software engineer', 'lead software engineer',
        'principal software engineer', 'staff software engineer'
    ]
    
    return any(keyword in title_lower for keyword in se_keywords)

def identify_tech_worker(row):
    """Identify if someone is a tech worker (broader definition)"""
    job_title = str(row['job_title']).lower() if pd.notna(row['job_title']) else ''
    industry = str(row['industry']).lower() if pd.notna(row['industry']) else ''
    
    # Industry-based identification
    tech_industries = [
        'computing or tech', 'software', 'tech', 'computing', 'it'
    ]
    
    if any(tech_ind in industry for tech_ind in tech_industries):
        return True
    
    # Job title-based identification
    tech_job_keywords = [
        'software', 'developer', 'engineer', 'programmer', 'data scientist',
        'data analyst', 'data engineer', 'devops', 'system admin', 'network',
        'security engineer', 'qa engineer', 'test engineer', 'technical',
        'it specialist', 'web developer', 'mobile developer', 'backend',
        'frontend', 'full stack', 'machine learning', 'ai engineer'
    ]
    
    if any(keyword in job_title for keyword in tech_job_keywords):
        return True
        
    return False

# Apply classifications
df_clean['is_software_engineer'] = df_clean['job_title'].apply(identify_software_engineer)
df_clean['is_tech_worker'] = df_clean.apply(identify_tech_worker, axis=1)

print("Tech worker identification results:")
print(f"Software Engineers: {df_clean['is_software_engineer'].sum():,}")
print(f"Tech Workers (broad): {df_clean['is_tech_worker'].sum():,}")
print(f"Percentage of dataset that are tech workers: {df_clean['is_tech_worker'].mean()*100:.1f}%")

# Focus on US tech workers for our analysis
us_tech = df_clean[(df_clean['country_clean'] == 'United States') & (df_clean['is_tech_worker'] == True)].copy()
us_se = df_clean[(df_clean['country_clean'] == 'United States') & (df_clean['is_software_engineer'] == True)].copy()

print(f"\nUS Tech Workers: {len(us_tech):,}")
print(f"US Software Engineers: {len(us_se):,}")

print("\nSample Software Engineer job titles:")
se_titles = df_clean[df_clean['is_software_engineer'] == True]['job_title'].value_counts().head(10)
print(se_titles)


JOB TITLE AND TECH WORKER IDENTIFICATION
Step 8: Identifying tech workers and software engineers...
Tech worker identification results:
Software Engineers: 1,212
Tech Workers (broad): 10,860
Percentage of dataset that are tech workers: 38.7%

US Tech Workers: 9,033
US Software Engineers: 969

Sample Software Engineer job titles:
job_title
Software Engineer              286
Senior Software Engineer       196
Software Developer              88
Software engineer               52
Staff Software Engineer         36
Software Engineer II            31
Principal Software Engineer     30
Senior Software Developer       30
Senior software engineer        25
Software developer              22
Name: count, dtype: int64
Tech worker identification results:
Software Engineers: 1,212
Tech Workers (broad): 10,860
Percentage of dataset that are tech workers: 38.7%

US Tech Workers: 9,033
US Software Engineers: 969

Sample Software Engineer job titles:
job_title
Software Engineer              286
Senior

In [ ]:
# DATA CLEANING SUMMARY
print("\n" + "=" * 60)
print("DATA CLEANING SUMMARY")
print("=" * 60)
print("🎉 ALL DATA CLEANING PHASES COMPLETED!")
print("")
print("📊 CLEANED DATASET OVERVIEW:")
print(f"   • Total records: {len(df_clean):,}")
print(f"   • Records with clean salaries: {df_clean['salary_usd_clean'].notna().sum():,} (99.0%)")
print(f"   • US records: {(df_clean['country_clean'] == 'United States').sum():,} (83.7%)")
print(f"   • Tech workers identified: {df_clean['is_tech_worker'].sum():,} (38.7%)")
print(f"   • Software engineers identified: {df_clean['is_software_engineer'].sum():,}")
print("")
print("🔧 CLEANING TRANSFORMATIONS APPLIED:")
print("   ✅ Salary: String → Numeric → USD conversion → Outlier removal")
print("   ✅ Country: 79 US variations → 'United States'") 
print("   ✅ States: Abbreviations → Full names (50 states + DC)")
print("   ✅ Experience: Ranges → Numeric midpoints")
print("   ✅ Education: Standardized 6 categories")
print("   ✅ Job titles: Tech worker & Software Engineer identification")
print("")
print("🎯 READY FOR BUSINESS QUESTIONS ANALYSIS!")
print("   Next: Answer the 4 core questions with clean data")

# Quick validation of our cleaned data
print("\n📈 CLEANED DATA VALIDATION:")
print(f"Salary range: ${df_clean['salary_usd_clean'].min():,.0f} - ${df_clean['salary_usd_clean'].max():,.0f}")
print(f"Median salary (all): ${df_clean['salary_usd_clean'].median():,.0f}")
print(f"US Software Engineers with salary data: {len(us_se[us_se['salary_usd_clean'].notna()]):,}")
print(f"US Tech Workers with salary data: {len(us_tech[us_tech['salary_usd_clean'].notna()]):,}")


DATA CLEANING SUMMARY
🎉 ALL DATA CLEANING PHASES COMPLETED!

📊 CLEANED DATASET OVERVIEW:
   • Total records: 28,062
   • Records with clean salaries: 27,783 (99.0%)
   • US records: 23,493 (83.7%)
   • Tech workers identified: 10,860 (38.7%)
   • Software engineers identified: 1,212

🔧 CLEANING TRANSFORMATIONS APPLIED:
   ✅ Salary: String → Numeric → USD conversion → Outlier removal
   ✅ Country: 79 US variations → 'United States'
   ✅ States: Abbreviations → Full names (50 states + DC)
   ✅ Experience: Ranges → Numeric midpoints
   ✅ Education: Standardized 6 categories
   ✅ Job titles: Tech worker & Software Engineer identification

🎯 READY FOR BUSINESS QUESTIONS ANALYSIS!
   Next: Answer the 4 core questions with clean data

📈 CLEANED DATA VALIDATION:
Salary range: $10,000 - $500,000
Median salary (all): $75,000
US Software Engineers with salary data: 959
US Tech Workers with salary data: 8,980


In [ ]:
# Load the dataset
df = pd.read_csv('Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv', sep='\t')

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {len(df.columns)}")
print("\nColumn names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

Dataset loaded successfully!
Shape: (28062, 18)
Columns: 18

Column names:
 1. Timestamp
 2. How old are you?
 3. What industry do you work in?
 4. Job title
 5. If your job title needs additional context, please clarify here:
 6. What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)
 7. How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.
 8. Please indicate the currency
 9. If "Other," please indicate the currency here: 
10. If your income needs additional context, please provide it here:
11. What country do you work in?
12. If you're in the U.S., what state do you work in?
13. What city do you work in?
14. How many years of professional work experience do you have overall?
15

In [ ]:
# Create shorter column names for easier analysis
df.columns = [
    'timestamp', 'age', 'industry', 'job_title', 'job_context', 
    'salary', 'additional_compensation', 'currency', 'other_currency', 
    'income_context', 'country', 'state', 'city', 'total_experience',
    'field_experience', 'education', 'gender', 'race'
]

print("Renamed columns for easier analysis:")
print(df.columns.tolist())
print(f"\nDataset shape: {df.shape}")
print(f"Total records: {len(df):,}")

# Display basic info about the dataset
print("\n" + "="*50)
print("BASIC DATASET INFO")
print("="*50)
df.info()

Renamed columns for easier analysis:
['timestamp', 'age', 'industry', 'job_title', 'job_context', 'salary', 'additional_compensation', 'currency', 'other_currency', 'income_context', 'country', 'state', 'city', 'total_experience', 'field_experience', 'education', 'gender', 'race']

Dataset shape: (28062, 18)
Total records: 28,062

BASIC DATASET INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28062 entries, 0 to 28061
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                28062 non-null  object 
 1   age                      28062 non-null  object 
 2   industry                 27988 non-null  object 
 3   job_title                28061 non-null  object 
 4   job_context              7262 non-null   object 
 5   salary                   28062 non-null  object 
 6   additional_compensation  20766 non-null  float64
 7   currency                 28062 non-null  object 

In [ ]:
# Examine first few rows
print("FIRST 3 ROWS OF DATA:")
print("="*60)
for i in range(3):
    print(f"\nROW {i+1}:")
    for col in df.columns:
        print(f"  {col}: {df.iloc[i][col]}")

# Look at key columns we'll need for analysis
print("\n" + "="*60)
print("SAMPLE OF KEY COLUMNS:")
print("="*60)
key_columns = ['salary', 'currency', 'job_title', 'industry', 'country', 'state', 'total_experience']
print(df[key_columns].head(10))

FIRST 3 ROWS OF DATA:

ROW 1:
  timestamp: 4/27/2021 11:02:10
  age: 25-34
  industry: Education (Higher Education)
  job_title: Research and Instruction Librarian
  job_context: nan
  salary: 55,000
  additional_compensation: 0.0
  currency: USD
  other_currency: nan
  income_context: nan
  country: United States
  state: Massachusetts
  city: Boston
  total_experience: 5-7 years
  field_experience: 5-7 years
  education: Master's degree
  gender: Woman
  race: White

ROW 2:
  timestamp: 4/27/2021 11:02:22
  age: 25-34
  industry: Computing or Tech
  job_title: Change & Internal Communications Manager
  job_context: nan
  salary: 54,600
  additional_compensation: 4000.0
  currency: GBP
  other_currency: nan
  income_context: nan
  country: United Kingdom
  state: nan
  city: Cambridge
  total_experience: 8 - 10 years
  field_experience: 5-7 years
  education: College degree
  gender: Non-binary
  race: White

ROW 3:
  timestamp: 4/27/2021 11:02:38
  age: 25-34
  industry: Accounting, 

In [ ]:
# Summary of initial exploration
print("INITIAL DATA EXPLORATION SUMMARY:")
print("="*50)
print(f"✓ Dataset loaded: {len(df):,} records with {len(df.columns)} columns")
print(f"✓ Key columns identified for analysis:")
print(f"  - Salary: {df['salary'].dtype} (contains commas, needs cleaning)")
print(f"  - Currency: {df['currency'].nunique()} unique currencies")
print(f"  - Job titles: {df['job_title'].nunique():,} unique job titles")
print(f"  - Industries: {df['industry'].nunique()} unique industries")
print(f"  - Countries: {df['country'].nunique()} unique countries")
print(f"  - US States: {df['state'].nunique()} unique states/values")
print(f"  - Experience: {df['total_experience'].nunique()} unique experience levels")

print(f"\n✓ Data quality observations:")
print(f"  - Salary column is string type with commas (needs conversion)")
print(f"  - Multiple currencies present (USD, GBP, others)")
print(f"  - Country names inconsistent (US, USA, United States)")
print(f"  - Experience data in ranges (needs numeric conversion)")
print(f"  - Missing values in several columns")

print("\nNext: Data Quality Assessment")
print("Ready to proceed to detailed data quality analysis!")

INITIAL DATA EXPLORATION SUMMARY:
✓ Dataset loaded: 28,062 records with 18 columns
✓ Key columns identified for analysis:
  - Salary: object (contains commas, needs cleaning)
  - Currency: 11 unique currencies
  - Job titles: 14,348 unique job titles
  - Industries: 1219 unique industries
  - Countries: 379 unique countries
  - US States: 134 unique states/values
  - Experience: 8 unique experience levels

✓ Data quality observations:
  - Salary column is string type with commas (needs conversion)
  - Multiple currencies present (USD, GBP, others)
  - Country names inconsistent (US, USA, United States)
  - Experience data in ranges (needs numeric conversion)
  - Missing values in several columns

Next: Data Quality Assessment
Ready to proceed to detailed data quality analysis!


In [ ]:
# DETAILED DATA QUALITY ASSESSMENT
print("=" * 60)
print("DATA QUALITY ASSESSMENT")
print("=" * 60)

# 1. Missing Values Analysis
print("\n1. MISSING VALUES ANALYSIS:")
print("-" * 30)
missing_counts = df.isnull().sum()
missing_pct = (missing_counts / len(df) * 100).round(2)
missing_summary = pd.DataFrame({
    'Missing_Count': missing_counts,
    'Missing_Percentage': missing_pct
}).sort_values('Missing_Count', ascending=False)

print(missing_summary[missing_summary['Missing_Count'] > 0])

# 2. Currency Analysis
print("\n2. CURRENCY DISTRIBUTION:")
print("-" * 30)
currency_counts = df['currency'].value_counts()
print(currency_counts)
print(f"\nOther currencies specified: {df['other_currency'].dropna().nunique()} unique values")
if len(df['other_currency'].dropna()) > 0:
    print("Sample other currencies:", df['other_currency'].dropna().head().tolist())

DATA QUALITY ASSESSMENT

1. MISSING VALUES ANALYSIS:
------------------------------
                         Missing_Count  Missing_Percentage
other_currency                   27856               99.27
income_context                   25020               89.16
job_context                      20800               74.12
additional_compensation           7296               26.00
state                             5023               17.90
education                          222                0.79
race                               177                0.63
gender                             171                0.61
city                                82                0.29
industry                            74                0.26
job_title                            1                0.00

2. CURRENCY DISTRIBUTION:
------------------------------
currency
USD        23374
CAD         1673
GBP         1591
EUR          643
AUD/NZD      504
Other        160
CHF           37
SEK           37
JPY  

In [ ]:
# 3. Salary Data Quality Issues
print("\n3. SALARY DATA QUALITY:")
print("-" * 30)
print(f"Sample salary values:")
print(df['salary'].head(10).tolist())
print(f"\nSalary data type: {df['salary'].dtype}")
print(f"Unique salary values (first 20): {sorted(df['salary'].unique())[:20]}")

# Check for problematic salary entries
problematic_salaries = df[df['salary'].str.contains(r'[a-zA-Z]', na=False, regex=True)]
print(f"\nSalaries containing letters: {len(problematic_salaries)}")
if len(problematic_salaries) > 0:
    print("Sample problematic salaries:")
    print(problematic_salaries[['salary', 'currency', 'job_title', 'country']].head())

# 4. Country Data Consistency
print("\n4. COUNTRY DATA CONSISTENCY:")
print("-" * 30)
country_counts = df['country'].value_counts().head(15)
print(country_counts)

# Check for US variations
us_variations = df[df['country'].str.contains(r'US|United States|America', na=False, case=False, regex=True)]['country'].value_counts()
print(f"\nUS country name variations:")
print(us_variations)


3. SALARY DATA QUALITY:
------------------------------
Sample salary values:
['55,000', '54,600', '34,000', '62,000', '60,000', '62,000', '33,000', '50,000', '112,000', '45,000']

Salary data type: object
Unique salary values (first 20): ['0', '00', '1', '1,000', '1,050,000', '1,080,000', '1,100,000', '1,150,000', '1,200,000', '1,260,000', '1,300,000', '1,324,000', '1,334,782', '1,360,000', '1,400,000', '1,442', '1,650,000', '1,900,000', '10,000', '10,500']

Salaries containing letters: 0

4. COUNTRY DATA CONSISTENCY:
------------------------------
country
United States               8992
USA                         7934
US                          2610
Canada                      1570
United States                667
U.S.                         580
UK                           574
United Kingdom               547
USA                          468
Usa                          448
United States of America     428
Australia                    318
United states                208
usa    

In [ ]:
# 5. Experience Data Analysis
print("\n5. EXPERIENCE DATA:")
print("-" * 30)
print("Total experience values:")
print(df['total_experience'].value_counts())

print("\nField experience values:")
print(df['field_experience'].value_counts())

# 6. Industry Categories (key for tech identification)
print("\n6. INDUSTRY ANALYSIS (Top 15):")
print("-" * 30)
print(df['industry'].value_counts().head(15))

# Look for tech-related industries
tech_keywords = ['tech', 'computing', 'software', 'IT', 'computer', 'engineering']
print(f"\nTech-related industries (containing: {tech_keywords}):")
tech_industries = df['industry'].dropna()
for keyword in tech_keywords:
    matching = tech_industries[tech_industries.str.contains(keyword, case=False, na=False)]
    if len(matching) > 0:
        print(f"\n'{keyword}' industries:")
        print(matching.value_counts().head())


5. EXPERIENCE DATA:
------------------------------
Total experience values:
total_experience
11 - 20 years       9624
8 - 10 years        5377
5-7 years           4882
21 - 30 years       3637
2 - 4 years         3026
31 - 40 years        869
1 year or less       523
41 years or more     124
Name: count, dtype: int64

Field experience values:
field_experience
11 - 20 years       6536
5-7 years           6519
2 - 4 years         6249
8 - 10 years        4982
21 - 30 years       1868
1 year or less      1485
31 - 40 years        382
41 years or more      41
Name: count, dtype: int64

6. INDUSTRY ANALYSIS (Top 15):
------------------------------
industry
Computing or Tech                       4699
Education (Higher Education)            2464
Nonprofits                              2419
Health care                             1896
Government and Public Administration    1889
Accounting, Banking & Finance           1809
Engineering or Manufacturing            1695
Marketing, Advertising &

In [ ]:
# 7. Job Title Analysis for Software Engineers
print("\n7. SOFTWARE ENGINEER JOB TITLE VARIATIONS:")
print("-" * 40)
software_engineer_titles = df['job_title'].dropna()
se_keywords = ['software engineer', 'software developer', 'developer', 'engineer', 'programmer']

print("Job titles containing software engineer terms:")
for keyword in se_keywords:
    matching = software_engineer_titles[software_engineer_titles.str.contains(keyword, case=False, na=False)]
    if len(matching) > 0:
        print(f"\n'{keyword}' ({len(matching)} total):")
        top_matches = matching.value_counts().head(5)
        print(top_matches)

# DATA QUALITY SUMMARY
print("\n" + "=" * 60)
print("DATA QUALITY ASSESSMENT SUMMARY")
print("=" * 60)
print("🔍 KEY FINDINGS:")
print("✓ 28,062 total records with 18 columns")
print("⚠️  Major cleaning needed:")
print("   • Salary: String format with commas, needs conversion to numeric")
print("   • Currency: 11 different currencies, majority USD (83%)")
print("   • Country: 79 variations of 'United States' alone")
print("   • Experience: Range format (e.g., '5-7 years') needs numeric conversion")
print("   • Job titles: 14,348 unique titles, many variations for same role")
print("   • Missing data: High in job_context (74%), moderate in state (18%)")
print("")
print("🎯 TECH IDENTIFICATION STRATEGY:")
print("   • Primary: 'Computing or Tech' industry (4,699 records)")
print("   • Secondary: Job titles with engineer/developer/programmer keywords")
print("   • Engineering industry: 1,695 additional potential tech workers")
print("")
print("🚀 READY FOR DATA CLEANING PHASE!")
print("   Next: Start with salary standardization → currency conversion → geographic cleaning")


7. SOFTWARE ENGINEER JOB TITLE VARIATIONS:
----------------------------------------
Job titles containing software engineer terms:

'software engineer' (949 total):
job_title
Software Engineer           286
Senior Software Engineer    196
Software engineer            52
Staff Software Engineer      36
Software Engineer II         31
Name: count, dtype: int64

'software developer' (208 total):
job_title
Software Developer           88
Senior Software Developer    30
Software developer           22
Senior software developer     8
Software developer            6
Name: count, dtype: int64

'developer' (505 total):
job_title
Software Developer           88
Senior Software Developer    30
Web Developer                25
Software developer           22
Senior Developer             19
Name: count, dtype: int64

'engineer' (2553 total):
job_title
Software Engineer           286
Senior Software Engineer    196
Engineering Manager          88
Software engineer            52
Senior Engineer      

## Step 3: Business Questions Analysis

Now answer those important business questions!


In [ ]:
# CORE QUESTION 1: What is the median salary for Software Engineers in the United States?
print("=" * 80)
print("CORE QUESTION 1: MEDIAN SALARY FOR SOFTWARE ENGINEERS IN US")
print("=" * 80)

# Filter for US Software Engineers with clean salary data
us_software_engineers = df_clean[
    (df_clean['country_clean'] == 'United States') & 
    (df_clean['is_software_engineer'] == True) & 
    (df_clean['salary_usd_clean'].notna())
].copy()

print(f"US Software Engineers with salary data: {len(us_software_engineers):,}")

if len(us_software_engineers) > 0:
    # Calculate median salary
    median_salary = us_software_engineers['salary_usd_clean'].median()
    mean_salary = us_software_engineers['salary_usd_clean'].mean()
    
    print(f"\n📊 RESULTS:")
    print(f"   Median Salary: ${median_salary:,.0f}")
    print(f"   Mean Salary:   ${mean_salary:,.0f}")
    
    # Additional statistics for validation
    print(f"\n📈 ADDITIONAL STATISTICS:")
    stats = us_software_engineers['salary_usd_clean'].describe()
    print(f"   Count: {stats['count']:.0f}")
    print(f"   Min:   ${stats['min']:,.0f}")
    print(f"   25th:  ${stats['25%']:,.0f}")
    print(f"   50th:  ${stats['50%']:,.0f} (Median)")
    print(f"   75th:  ${stats['75%']:,.0f}")
    print(f"   Max:   ${stats['max']:,.0f}")
    
    # Sample of job titles to validate
    print(f"\n🔍 VALIDATION - Sample Job Titles:")
    title_sample = us_software_engineers['job_title'].value_counts().head(8)
    for title, count in title_sample.items():
        print(f"   • {title}: {count} people")
        
    print(f"\n✅ ANSWER TO QUESTION 1:")
    print(f"   The median salary for Software Engineers in the United States is ${median_salary:,.0f}")
    
else:
    print("❌ No Software Engineers found in US data")

CORE QUESTION 1: MEDIAN SALARY FOR SOFTWARE ENGINEERS IN US
US Software Engineers with salary data: 959

📊 RESULTS:
   Median Salary: $140,000
   Mean Salary:   $145,320

📈 ADDITIONAL STATISTICS:
   Count: 959
   Min:   $10,700
   25th:  $110,000
   50th:  $140,000 (Median)
   75th:  $172,000
   Max:   $500,000

🔍 VALIDATION - Sample Job Titles:
   • Software Engineer: 234 people
   • Senior Software Engineer: 167 people
   • Software Developer: 56 people
   • Software engineer: 37 people
   • Staff Software Engineer: 33 people
   • Software Engineer II: 31 people
   • Principal Software Engineer: 25 people
   • Senior Software Developer: 18 people

✅ ANSWER TO QUESTION 1:
   The median salary for Software Engineers in the United States is $140,000


In [ ]:
# CORE QUESTION 2: Which US state has the highest average salary for tech workers?
print("=" * 80)
print("CORE QUESTION 2: US STATE WITH HIGHEST AVERAGE TECH SALARY")
print("=" * 80)

# Filter for US tech workers with clean salary and state data
us_tech_with_state = df_clean[
    (df_clean['country_clean'] == 'United States') & 
    (df_clean['is_tech_worker'] == True) & 
    (df_clean['salary_usd_clean'].notna()) &
    (df_clean['state_clean'].notna())
].copy()

print(f"US Tech Workers with salary and state data: {len(us_tech_with_state):,}")

# Calculate average salary by state (minimum 20 workers for reliability)
min_workers = 20
state_salary_stats = us_tech_with_state.groupby('state_clean')['salary_usd_clean'].agg([
    'count', 'mean', 'median', 'std'
]).round(0)

# Filter states with sufficient sample size
reliable_states = state_salary_stats[state_salary_stats['count'] >= min_workers].copy()
reliable_states = reliable_states.sort_values('mean', ascending=False)

print(f"\nStates with at least {min_workers} tech workers: {len(reliable_states)}")

print(f"\n🏆 TOP 15 STATES BY AVERAGE TECH SALARY:")
print("   Rank | State                    | Workers | Average   | Median   ")
print("   -----|--------------------------|---------|-----------|----------")

for i, (state, stats) in enumerate(reliable_states.head(15).iterrows(), 1):
    print(f"   {i:2d}   | {state:<24} | {stats['count']:3.0f}     | ${stats['mean']:7.0f} | ${stats['median']:7.0f}")

# Get the winner
if len(reliable_states) > 0:
    top_state = reliable_states.index[0]
    top_avg = reliable_states.iloc[0]['mean']
    top_count = reliable_states.iloc[0]['count']
    
    print(f"\n✅ ANSWER TO QUESTION 2:")
    print(f"   {top_state} has the highest average salary for tech workers")
    print(f"   Average: ${top_avg:,.0f} (based on {top_count:.0f} workers)")
    
    # Show some context
    print(f"\n📊 COMPARISON WITH OTHER TOP STATES:")
    for i, (state, stats) in enumerate(reliable_states.head(5).iterrows(), 1):
        print(f"   {i}. {state}: ${stats['mean']:,.0f} avg (n={stats['count']:.0f})")

else:
    print("❌ No states found with sufficient tech worker data")

CORE QUESTION 2: US STATE WITH HIGHEST AVERAGE TECH SALARY
US Tech Workers with salary and state data: 8,799

States with at least 20 tech workers: 42

🏆 TOP 15 STATES BY AVERAGE TECH SALARY:
   Rank | State                    | Workers | Average   | Median   
   -----|--------------------------|---------|-----------|----------
    1   | California               | 1203     | $ 130265 | $ 122000
    2   | Washington               | 594     | $ 120293 | $ 113752
    3   | New York                 | 978     | $ 109200 | $  94700
    4   | Massachusetts            | 680     | $ 107805 | $ 100000
    5   | Oregon                   | 269     | $ 105392 | $  94000
    6   | Colorado                 | 290     | $ 102688 | $  94000
    7   | Virginia                 | 289     | $ 102167 | $  91000
    8   | New Mexico               |  30     | $  99316 | $  90000
    9   | Maryland                 | 179     | $  98267 | $  95000
   10   | Texas                    | 469     | $  97139 | $  90000

In [ ]:
# CORE QUESTION 3: How much does salary increase on average for each year of experience in tech?
print("=" * 80)
print("CORE QUESTION 3: SALARY INCREASE PER YEAR OF EXPERIENCE IN TECH")
print("=" * 80)

# Filter for tech workers with clean salary and experience data
tech_exp_data = df_clean[
    (df_clean['is_tech_worker'] == True) & 
    (df_clean['salary_usd_clean'].notna()) &
    (df_clean['total_experience_numeric'].notna())
].copy()

print(f"Tech workers with salary and experience data: {len(tech_exp_data):,}")

# Calculate correlation and regression
from scipy import stats
import numpy as np

experience = tech_exp_data['total_experience_numeric']
salary = tech_exp_data['salary_usd_clean']

# Calculate correlation
correlation = experience.corr(salary)
print(f"\n📊 CORRELATION ANALYSIS:")
print(f"   Correlation coefficient: {correlation:.3f}")

# Linear regression to find salary increase per year
slope, intercept, r_value, p_value, std_err = stats.linregress(experience, salary)

print(f"\n📈 LINEAR REGRESSION RESULTS:")
print(f"   Slope (salary increase per year): ${slope:,.0f}")
print(f"   Intercept: ${intercept:,.0f}")
print(f"   R-squared: {r_value**2:.3f}")
print(f"   P-value: {p_value:.2e}")

# Show salary by experience brackets for validation
print(f"\n🔍 VALIDATION - Average Salary by Experience Level:")
exp_brackets = tech_exp_data.groupby('total_experience_numeric')['salary_usd_clean'].agg([
    'count', 'mean', 'median'
]).round(0)

for exp_years, stats in exp_brackets.iterrows():
    if stats['count'] >= 10:  # Only show brackets with sufficient data
        print(f"   {exp_years:2.0f} years: ${stats['mean']:6.0f} avg, ${stats['median']:6.0f} median (n={stats['count']:.0f})")

# Calculate practical examples
entry_level_salary = intercept + slope * 2  # 2 years experience
senior_level_salary = intercept + slope * 15  # 15 years experience
difference = senior_level_salary - entry_level_salary

print(f"\n💰 PRACTICAL EXAMPLES:")
print(f"   Predicted salary at 2 years:  ${entry_level_salary:,.0f}")
print(f"   Predicted salary at 15 years: ${senior_level_salary:,.0f}")
print(f"   Difference over 13 years:     ${difference:,.0f}")
print(f"   Average increase per year:    ${difference/13:,.0f}")

print(f"\n✅ ANSWER TO QUESTION 3:")
print(f"   Tech worker salaries increase by approximately ${slope:,.0f} per year of experience")
print(f"   This represents a {slope/tech_exp_data['salary_usd_clean'].mean()*100:.1f}% increase relative to average salary")

CORE QUESTION 3: SALARY INCREASE PER YEAR OF EXPERIENCE IN TECH
Tech workers with salary and experience data: 10,750

📊 CORRELATION ANALYSIS:
   Correlation coefficient: 0.231

📈 LINEAR REGRESSION RESULTS:
   Slope (salary increase per year): $1,503
   Intercept: $78,867
   R-squared: 0.053
   P-value: 6.36e-130

🔍 VALIDATION - Average Salary by Experience Level:
    0 years: $ 69916 avg, $ 63000 median (n=171)
    3 years: $ 77667 avg, $ 68000 median (n=1214)
    6 years: $ 84770 avg, $ 75000 median (n=1947)
    9 years: $ 93074 avg, $ 81500 median (n=2060)
   15 years: $106773 avg, $ 97000 median (n=3679)
   25 years: $116891 avg, $107852 median (n=1380)
   35 years: $110322 avg, $104000 median (n=257)
   45 years: $106046 avg, $106875 median (n=42)

💰 PRACTICAL EXAMPLES:
   Predicted salary at 2 years:  $81,872
   Predicted salary at 15 years: $101,407
   Difference over 13 years:     $19,535
   Average increase per year:    $1,503

✅ ANSWER TO QUESTION 3:
   Tech worker salaries in

In [ ]:

# Question 4: What percentage of respondents work remotely vs. in-office?


In [ ]:
# CORE QUESTION 4: Which industry (besides tech) has the highest median salary?
print("=" * 80)
print("CORE QUESTION 4: HIGHEST PAYING NON-TECH INDUSTRY")
print("=" * 80)

# Filter out tech workers and get clean salary data
non_tech_workers = df_clean[
    (df_clean['is_tech_worker'] == False) & 
    (df_clean['salary_usd_clean'].notna()) &
    (df_clean['industry'].notna())
].copy()

print(f"Non-tech workers with salary and industry data: {len(non_tech_workers):,}")

# Calculate median salary by industry (minimum 50 workers for reliability)
min_workers = 50
industry_stats = non_tech_workers.groupby('industry')['salary_usd_clean'].agg([
    'count', 'median', 'mean', 'std'
]).round(0)

# Filter industries with sufficient sample size
reliable_industries = industry_stats[industry_stats['count'] >= min_workers].copy()
reliable_industries = reliable_industries.sort_values('median', ascending=False)

print(f"\nNon-tech industries with at least {min_workers} workers: {len(reliable_industries)}")

print(f"\n🏆 TOP 15 NON-TECH INDUSTRIES BY MEDIAN SALARY:")
print("   Rank | Industry                              | Workers | Median   | Average  ")
print("   -----|---------------------------------------|---------|----------|----------")

for i, (industry, stats) in enumerate(reliable_industries.head(15).iterrows(), 1):
    # Truncate long industry names
    industry_short = industry[:37] + "..." if len(industry) > 40 else industry
    print(f"   {i:2d}   | {industry_short:<37} | {stats['count']:3.0f}     | ${stats['median']:7.0f} | ${stats['mean']:7.0f}")

# Get the winner
if len(reliable_industries) > 0:
    top_industry = reliable_industries.index[0]
    top_median = reliable_industries.iloc[0]['median']
    top_count = reliable_industries.iloc[0]['count']
    
    print(f"\n✅ ANSWER TO QUESTION 4:")
    print(f"   '{top_industry}' has the highest median salary among non-tech industries")
    print(f"   Median: ${top_median:,.0f} (based on {top_count:.0f} workers)")
    
    # Show top 5 for context
    print(f"\n📊 TOP 5 NON-TECH INDUSTRIES:")
    for i, (industry, stats) in enumerate(reliable_industries.head(5).iterrows(), 1):
        print(f"   {i}. {industry}")
        print(f"      Median: ${stats['median']:,.0f}, Average: ${stats['mean']:,.0f} (n={stats['count']:.0f})")
        
    # Compare with tech median for reference
    tech_median = df_clean[df_clean['is_tech_worker'] == True]['salary_usd_clean'].median()
    print(f"\n🔄 COMPARISON:")
    print(f"   Top non-tech industry median: ${top_median:,.0f}")
    print(f"   Tech workers median:          ${tech_median:,.0f}")
    print(f"   Difference:                   ${tech_median - top_median:,.0f} (tech pays more)")

else:
    print("❌ No non-tech industries found with sufficient worker data")

CORE QUESTION 4: HIGHEST PAYING NON-TECH INDUSTRY
Non-tech workers with salary and industry data: 16,967

Non-tech industries with at least 50 workers: 21

🏆 TOP 15 NON-TECH INDUSTRIES BY MEDIAN SALARY:
   Rank | Industry                              | Workers | Median   | Average  
   -----|---------------------------------------|---------|----------|----------
    1   | Law                                   | 1084     | $  90000 | $ 113510
    2   | Business or Consulting                | 813     | $  88200 | $ 100233
    3   | Engineering or Manufacturing          | 855     | $  80000 | $  87648
    4   | Accounting, Banking & Finance         | 1716     | $  76000 | $  86787
    5   | Health care                           | 1802     | $  75000 | $  86849
    6   | Marketing, Advertising & PR           | 1087     | $  73000 | $  84988
    7   | Insurance                             | 480     | $  73000 | $  82658
    8   | Government and Public Administration  | 1796     | $  72715 |

In [ ]:
# BONUS QUESTIONS ANALYSIS
print("=" * 80)
print("BONUS QUESTIONS ANALYSIS")
print("=" * 80)

# BONUS QUESTION 1: Gender pay gap in tech roles
print("BONUS QUESTION 1: Gender Pay Gap in Tech Roles")
print("-" * 50)

tech_gender_data = df_clean[
    (df_clean['is_tech_worker'] == True) & 
    (df_clean['salary_usd_clean'].notna()) &
    (df_clean['gender'].notna()) &
    (df_clean['gender'].isin(['Man', 'Woman']))  # Focus on binary gender comparison
].copy()

gender_stats = tech_gender_data.groupby('gender')['salary_usd_clean'].agg([
    'count', 'median', 'mean', 'std'
]).round(0)

print(f"Tech workers by gender (with salary data):")
for gender, stats in gender_stats.iterrows():
    print(f"   {gender}: {stats['count']:.0f} people, Median: ${stats['median']:,.0f}, Mean: ${stats['mean']:,.0f}")

if len(gender_stats) >= 2:
    man_median = gender_stats.loc['Man', 'median']
    woman_median = gender_stats.loc['Woman', 'median']
    gap_absolute = man_median - woman_median
    gap_percentage = (gap_absolute / woman_median) * 100
    
    print(f"\n💰 GENDER PAY GAP ANALYSIS:")
    print(f"   Men median salary:     ${man_median:,.0f}")
    print(f"   Women median salary:   ${woman_median:,.0f}")
    print(f"   Gap (absolute):        ${gap_absolute:,.0f}")
    print(f"   Gap (percentage):      {gap_percentage:.1f}%")

# BONUS QUESTION 2: Master's vs Bachelor's degree salary difference
print(f"\n\nBONUS QUESTION 2: Master's vs Bachelor's Degree Salary Difference")
print("-" * 60)

education_data = df_clean[
    (df_clean['salary_usd_clean'].notna()) &
    (df_clean['education_clean'].isin(["Bachelor's Degree", "Master's Degree"]))
].copy()

education_stats = education_data.groupby('education_clean')['salary_usd_clean'].agg([
    'count', 'median', 'mean', 'std'
]).round(0)

print(f"Salary by education level:")
for education, stats in education_stats.iterrows():
    print(f"   {education}: {stats['count']:.0f} people, Median: ${stats['median']:,.0f}, Mean: ${stats['mean']:,.0f}")

if len(education_stats) >= 2:
    masters_median = education_stats.loc["Master's Degree", 'median']
    bachelors_median = education_stats.loc["Bachelor's Degree", 'median']
    edu_gap = masters_median - bachelors_median
    edu_gap_pct = (edu_gap / bachelors_median) * 100
    
    print(f"\n🎓 EDUCATION SALARY PREMIUM:")
    print(f"   Master's median:       ${masters_median:,.0f}")
    print(f"   Bachelor's median:     ${bachelors_median:,.0f}")
    print(f"   Premium (absolute):    ${edu_gap:,.0f}")
    print(f"   Premium (percentage):  {edu_gap_pct:.1f}%")

# Statistical significance test for education difference
from scipy.stats import mannwhitneyu

masters_salaries = education_data[education_data['education_clean'] == "Master's Degree"]['salary_usd_clean']
bachelors_salaries = education_data[education_data['education_clean'] == "Bachelor's Degree"]['salary_usd_clean']

statistic, p_value = mannwhitneyu(masters_salaries, bachelors_salaries, alternative='two-sided')
print(f"   Statistical significance (Mann-Whitney U): p = {p_value:.2e}")
print(f"   Result: {'Significant' if p_value < 0.05 else 'Not significant'} difference (α = 0.05)")

BONUS QUESTIONS ANALYSIS
BONUS QUESTION 1: Gender Pay Gap in Tech Roles
--------------------------------------------------
Tech workers by gender (with salary data):
   Man: 3082 people, Median: $110,000, Mean: $118,197
   Woman: 7169 people, Median: $80,000, Mean: $89,170

💰 GENDER PAY GAP ANALYSIS:
   Men median salary:     $110,000
   Women median salary:   $80,000
   Gap (absolute):        $30,000
   Gap (percentage):      37.5%


BONUS QUESTION 2: Master's vs Bachelor's Degree Salary Difference
------------------------------------------------------------
Salary by education level:
   Bachelor's Degree: 13419 people, Median: $72,000, Mean: $83,254
   Master's Degree: 8778 people, Median: $78,000, Mean: $88,389

🎓 EDUCATION SALARY PREMIUM:
   Master's median:       $78,000
   Bachelor's median:     $72,000
   Premium (absolute):    $6,000
   Premium (percentage):  8.3%
   Statistical significance (Mann-Whitney U): p = 6.55e-32
   Result: Significant difference (α = 0.05)


## Final Summary

**Summarize your findings here:**

1. **Median salary for Software Engineers in US:** $140,000
2. **Highest paying US state for tech:** California ($130,265 average)
3. **Salary increase per year of experience:** $1,503 per year
4. **Highest paying non-tech industry:** Law ($90,000 median)

**Key insights:**
- **Tech pays well but Law leads non-tech industries:** While tech workers have strong salaries, the Law industry actually has the highest median salary among non-tech sectors at $90,000, which is $4,000 higher than the overall tech median of $86,000
- **Geographic concentration drives premium:** California dominates tech salaries with an average of $130,265, followed by Washington at $120,293. The top 5 states (CA, WA, NY, MA, OR) all exceed $105,000 average, showing clear geographic clustering of high-paying tech roles
- **Experience premium is moderate but consistent:** Each year of experience adds approximately $1,503 to salary (1.5% increase), with clear progression from entry-level (~$70K) to senior levels (~$107K), though the correlation is moderate (R² = 0.053) indicating other factors significantly influence compensation

**Challenges faced:**
- llm might be incorrect so i solved it by verifying it with the sources (ie reading docs and other articles)

**What you learned about vibe coding:**
- it good to certain extend using ai and apporval the changes can be chaos 
